#Carregando bibliotecas


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



#Carregando dados


In [ ]:
df = pd.read_excel('/content/celularessubtraidos.xlsx', dtype={'CEP': str})

#Limpeza de dados

In [ ]:
# Preencher coluna CEP com zeros a esquerda até completar os 8 digitos
df['CEP'] = df['CEP'].str.zfill(8)

In [ ]:
#Informações gerais sobre os dados (Colunas e tipos de dados)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266333 entries, 0 to 266332
Data columns (total 51 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ID_DELEGACIA            266333 non-null  int64         
 1   NOME_DEPARTAMENTO       266333 non-null  object        
 2   NOME_SECCIONAL          266333 non-null  object        
 3   NOME_DELEGACIA          266333 non-null  object        
 4   NOME_MUNICIPIO          266333 non-null  object        
 5   ANO_BO                  266333 non-null  int64         
 6   NUM_BO                  266333 non-null  object        
 7   VERSAO                  266333 non-null  int64         
 8   NOME_DEPARTAMENTO_CIRC  266333 non-null  object        
 9   NOME_SECCIONAL_CIRC     266333 non-null  object        
 10  NOME_DELEGACIA_CIRC     266333 non-null  object        
 11  NOME_MUNICIPIO_CIRC     266333 non-null  object        
 12  DATA_OCORRENCIA_BO      266333

In [ ]:
#Verificação de valores nulos
valores_nulos = df.isnull().sum()
print(valores_nulos)

ID_DELEGACIA                   0
NOME_DEPARTAMENTO              0
NOME_SECCIONAL                 0
NOME_DELEGACIA                 0
NOME_MUNICIPIO                 0
ANO_BO                         0
NUM_BO                         0
VERSAO                         0
NOME_DEPARTAMENTO_CIRC         0
NOME_SECCIONAL_CIRC            0
NOME_DELEGACIA_CIRC            0
NOME_MUNICIPIO_CIRC            0
DATA_OCORRENCIA_BO             0
HORA_OCORRENCIA           116917
DESCRICAO_APRESENTACAO         0
DATAHORA_REGISTRO_BO           0
DATA_COMUNICACAO_BO            0
DATAHORA_IMPRESSAO_BO        137
DESCR_PERIODO             149416
AUTORIA_BO                     0
FLAG_INTOLERANCIA              0
TIPO_INTOLERANCIA         266318
FLAG_FLAGRANTE                 0
FLAG_STATUS                    0
DESC_LEI                       0
FLAG_ATO_INFRACIONAL           0
RUBRICA                        0
DESCR_CONDUTA              46551
DESDOBRAMENTO             258814
CIRCUNSTANCIA             215920
DESCR_TIPO

In [ ]:
#Verificação de valores nulos em percentual
valores_nulos_percentual = (df.isnull().sum() / len(df)) * 100
print(valores_nulos_percentual)

ID_DELEGACIA               0.000000
NOME_DEPARTAMENTO          0.000000
NOME_SECCIONAL             0.000000
NOME_DELEGACIA             0.000000
NOME_MUNICIPIO             0.000000
ANO_BO                     0.000000
NUM_BO                     0.000000
VERSAO                     0.000000
NOME_DEPARTAMENTO_CIRC     0.000000
NOME_SECCIONAL_CIRC        0.000000
NOME_DELEGACIA_CIRC        0.000000
NOME_MUNICIPIO_CIRC        0.000000
DATA_OCORRENCIA_BO         0.000000
HORA_OCORRENCIA           43.898803
DESCRICAO_APRESENTACAO     0.000000
DATAHORA_REGISTRO_BO       0.000000
DATA_COMUNICACAO_BO        0.000000
DATAHORA_IMPRESSAO_BO      0.051439
DESCR_PERIODO             56.101197
AUTORIA_BO                 0.000000
FLAG_INTOLERANCIA          0.000000
TIPO_INTOLERANCIA         99.994368
FLAG_FLAGRANTE             0.000000
FLAG_STATUS                0.000000
DESC_LEI                   0.000000
FLAG_ATO_INFRACIONAL       0.000000
RUBRICA                    0.000000
DESCR_CONDUTA             17

#Criando uma cópia para transformações - Tratando dados nulos

In [ ]:
df_copia = df.copy()

In [ ]:
# Corrigir os valores no formato HH:MM:SS.0000000 para HH:MM:SS
df_copia['HORA_OCORRENCIA'] = df_copia['HORA_OCORRENCIA'].apply(lambda x: str(x).split('.')[0] if isinstance(x, str) else x)


In [ ]:
#Mudando o tipo de dado da coluna "HORA_OCORRENCIA" para DateTime.
df_copia["HORA_OCORRENCIA"] = pd.to_datetime(df_copia["HORA_OCORRENCIA"], format='%H:%M:%S', errors='coerce').dt.time


In [ ]:
#Colunas "HORA_OCORRENCIA" e "DESCR_PERIODO" se complementam, é possivel preencher valores nulos relacionando ambas.
# Função para preencher "DESCR_PERIODO" com base na "HORA_OCORRENCIA".
def determinar_periodo(hora):
    if pd.isnull(hora):
        return "Em hora incerta"
    hora = pd.to_datetime(hora, format='%H:%M:%S').time()
    if hora >= pd.to_datetime("00:00:00", format='%H:%M:%S').time() and hora <= pd.to_datetime("05:59:59", format='%H:%M:%S').time():
        return "De madrugada"
    elif hora >= pd.to_datetime("06:00:00", format='%H:%M:%S').time() and hora <= pd.to_datetime("11:59:59", format='%H:%M:%S').time():
        return "Pela manhã"
    elif hora >= pd.to_datetime("12:00:00", format='%H:%M:%S').time() and hora <= pd.to_datetime("17:59:59", format='%H:%M:%S').time():
        return "A tarde"
    elif hora >= pd.to_datetime("18:00:00", format='%H:%M:%S').time() and hora <= pd.to_datetime("23:59:59", format='%H:%M:%S').time():
        return "A noite"
    else:
        return "Em hora incerta"



In [ ]:
# Exceutar função para preencher os valores ausentes de DESCR_PERIODO
df_copia["DESCR_PERIODO"] = df_copia.apply(
    lambda row: determinar_periodo(row["HORA_OCORRENCIA"]) if pd.isnull(row["DESCR_PERIODO"]) else row["DESCR_PERIODO"],
    axis=1
)


In [ ]:
# Função para preencher "HORA_OCORRENCIA" com base na "DESCR_PERIODO".
def mapear_hora(periodo):
    if periodo == "A noite":
        return "22:00:00"
    elif periodo == "A tarde":
        return "15:00:00"
    elif periodo == "De madrugada":
        return "03:00:00"
    elif periodo == "Pela manhã":
        return "09:00:00"
    else:
        return None

In [ ]:
#Aplicar função
df_copia['HORA_OCORRENCIA'] = df_copia.apply(
    lambda row: mapear_hora(row['DESCR_PERIODO']) if pd.isna(row['HORA_OCORRENCIA']) else row['HORA_OCORRENCIA'],
    axis=1
)

In [ ]:
#Excluir colunas com alto percentual de valores nulos
df_copia = df_copia.drop(columns=['TIPO_INTOLERANCIA', 'DESDOBRAMENTO', 'CIRCUNSTANCIA', 'DESCR_UNIDADE'])

In [ ]:
# Excluir linhas onde as colunas especificadas possuem menos de 5% de valores nulos.
df_copia = df_copia.dropna(subset=['HORA_OCORRENCIA', 'DATAHORA_IMPRESSAO_BO', 'DESCR_TIPOLOCAL', 'DESCR_SUBTIPOLOCAL', 'BAIRRO', 'MARCA_OBJETO'])

In [ ]:
# Coluna 'DESCR_CONDUTA' substituir por Não especificado.
df_copia['DESCR_CONDUTA'] = df_copia['DESCR_CONDUTA'].fillna('Não especificado')

In [ ]:
# Coluna 'NUMERO_LOGRADOURO' substituir por valor padrão '10' uma vez que o numero não é importante para análise, mas sim, a região.
df_copia['NUMERO_LOGRADOURO'] = df_copia['NUMERO_LOGRADOURO'].fillna('10')

In [ ]:
# Contando valores 'FLAG_BLOQUEIO' e 'FLAG_DESBLOQUEIO'
contagem1 = df_copia['FLAG_BLOQUEIO'].value_counts(dropna=False)
contagem2 = df_copia['FLAG_DESBLOQUEIO'].value_counts(dropna=False)

# Exibir contagem
print(contagem1)
print(contagem2)

FLAG_BLOQUEIO
S      163994
N       70646
NaN     18377
Name: count, dtype: int64
FLAG_DESBLOQUEIO
N      234253
NaN     18377
S         387
Name: count, dtype: int64


In [ ]:
# Optar pela inclusão da MODA.
moda_bloqueio = df_copia['FLAG_BLOQUEIO'].mode()[0]
moda_desbloqueio = df_copia['FLAG_DESBLOQUEIO'].mode()[0]

df_copia['FLAG_DESBLOQUEIO'] = df_copia['FLAG_DESBLOQUEIO'].fillna(moda_desbloqueio)
df_copia['FLAG_BLOQUEIO'] = df_copia['FLAG_BLOQUEIO'].fillna(moda_bloqueio)

In [ ]:
# Coluna 'CEP' substituir por valor padrão '99999999' temos colunas mais importantes para análise, metodo api não seria viavel.
df_copia['CEP'] = df_copia['CEP'].fillna('99999999')

In [ ]:
# Calcular a média de LATITUDE e LONGITUDE para cada CIDADE (Poderia usar bairros, mas existem bairros sem lat e lon)
medias = df_copia.groupby('CIDADE')[['LATITUDE', 'LONGITUDE']].mean()

# Substituir os valores nulos na coluna LATITUDE
df_copia['LATITUDE'] = df_copia.apply(
    lambda row: medias.loc[row['CIDADE'], 'LATITUDE'] if pd.isnull(row['LATITUDE']) else row['LATITUDE'], axis=1
)

# Substituir os valores nulos na coluna LONGITUDE
df_copia['LONGITUDE'] = df_copia.apply(
    lambda row: medias.loc[row['CIDADE'], 'LONGITUDE'] if pd.isnull(row['LONGITUDE']) else row['LONGITUDE'], axis=1
    )

In [ ]:
# Verificar linhas duplicadas
duplicadas = df_copia.duplicated().sum()
print(f"Número de linhas duplicadas: {duplicadas}")

Número de linhas duplicadas: 18520


In [ ]:
# Excluindo linhas duplicadas
df_copia = df_copia.drop_duplicates()

In [ ]:
df_copia.info()


<class 'pandas.core.frame.DataFrame'>
Index: 234497 entries, 0 to 266332
Data columns (total 47 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ID_DELEGACIA            234497 non-null  int64         
 1   NOME_DEPARTAMENTO       234497 non-null  object        
 2   NOME_SECCIONAL          234497 non-null  object        
 3   NOME_DELEGACIA          234497 non-null  object        
 4   NOME_MUNICIPIO          234497 non-null  object        
 5   ANO_BO                  234497 non-null  int64         
 6   NUM_BO                  234497 non-null  object        
 7   VERSAO                  234497 non-null  int64         
 8   NOME_DEPARTAMENTO_CIRC  234497 non-null  object        
 9   NOME_SECCIONAL_CIRC     234497 non-null  object        
 10  NOME_DELEGACIA_CIRC     234497 non-null  object        
 11  NOME_MUNICIPIO_CIRC     234497 non-null  object        
 12  DATA_OCORRENCIA_BO      234497 non-

In [ ]:
df_copia.to_excel('arquivo_limpo.xlsx', index=False)